# Preprocessing data

### Standardization, or mean removal and variance scaling


Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn;

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.



In [1]:
from sklearn import preprocessing
import numpy as np
import pandas as pd

X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

X_scaled = preprocessing.scale(X_train)   # default axis=0 standardize each feature

X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

The standard score of a sample x is calculated as:
<tt>z = (x - u) / s</tt>
where u is the mean of the training samples or zero if <tt>with_mean=False</tt>, and s is the standard deviation of the training samples or one if <tt>with_std=False</tt>.


The preprocessing module further provides a utility class StandardScaler that implements the Transformer API to compute the mean and standard deviation on a training set so as to be able to later reapply the same transformation on the testing set.

In [2]:
sss = preprocessing.StandardScaler()
scaler = sss.fit(X_train)

In [3]:
scaler.transform(X_train)

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [4]:
scaler.transform(X_train).mean(), scaler.transform(X_train).std()

(np.float64(4.9343245538895844e-17), np.float64(1.0))

The scaler instance can then be used on new data to transform it the same way it did on the training set



In [5]:
X_test = np.array([[ 2., -3.,  2.],
                    [ 2.,  5.,  0.],
                    [ 0.,  1., -1.]])

In [6]:
scaler.transform(X_test)

array([[ 1.22474487, -3.67423461,  1.33630621],
       [ 1.22474487,  6.12372436, -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [7]:
scaler.fit(X_train)
scaler.transform(X_train)

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [8]:
scaler.fit_transform(X_train)

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

### Scaling features to a range

An alternative standardization is scaling features to lie between a given minimum and maximum value, often between zero and one, or so that the maximum absolute value of each feature is scaled to unit size.

In [9]:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

In [10]:
min_max_scaler = preprocessing.MinMaxScaler((0,1)) # default [0,1]

In [11]:
min_max_scaler.fit(X_train)

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [12]:
min_max_scaler.transform(X_train)

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

In [14]:
X_test = np.array([[-3., -1.,  4.]])
min_max_scaler.transform(X_test)

array([[-1.5       ,  0.        ,  1.66666667]])

The transformation is given by:

X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

X_scaled = X_std * (max - min) + min

If your data contains many outliers, scaling using the mean and variance of the data is likely to not work very well. In these cases, you can use robust_scale and RobustScaler (x_scaled = x / max(abs(x)))
as drop-in replacements instead. They use more robust estimates for the center and range of your data.



In [15]:
from sklearn.preprocessing import RobustScaler
X = [[ 1., -2.,  2.],
     [ -2.,  1.,  3.],
     [ 4.,  1., -2.]]

In [16]:
transformer = RobustScaler().fit(X)

In [17]:
transformer.scale_

array([3. , 1.5, 2.5])

In [18]:
transformer.transform(X)

array([[ 0. , -2. ,  0. ],
       [-1. ,  0. ,  0.4],
       [ 1. ,  0. , -1.6]])

The transformation is computed as:

<tt>X_scaled = (X - X.median(axis=0)) / X.IQR(axis=0)</tt>

where <tt>IQR = q3 - q1</tt>

The IQR is the range between the 1st quartile (q1) and the 3rd quartile (q3).

## Normalization


Normalization is the process of scaling individual samples to have unit norm. This process can be useful if you plan to use a quadratic form such as the dot-product or any other kernel to quantify the similarity of any pair of samples.

‘l1’
The l1 norm uses the sum of all the values and thus gives equal penalty to all parameters, enforcing sparsity.
x_normalized = x / sum(X)


‘l2’
The l2 norm uses the square root of the sum of all the squared values. This creates smoothness and rotational invariance. Some models, like PCA, assume rotational invariance, and so l2 will perform better.
x_normalized = x / sqrt(sum((i**2) for i in X))

In [4]:
import numpy as np
from sklearn import preprocessing
X = np.array([[1., -1.,  2.],
              [ 2.,  0.,  0.],
              [ 0.,  1., -1.]])
X_normalized = preprocessing.normalize(X, norm='l2')

In [5]:
# the same as previous X_normalized = preprocessing.Normalizer(norm='l2').fit_transform(X)

In [6]:
X_normalized

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [21]:
np.sqrt(0.40824829**2 + 0.40824829**2 + 0.81649658**2)

np.float64(0.9999999988637723)

In [22]:
np.sqrt(0**2 + 0.70710678**2 + (-0.70710678)**2)

np.float64(0.9999999983219684)

You can refer to [https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html) for more details.

##  Encoding categorical features

### OneHotEncoder

In [23]:
X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]
X

[['male', 'Spain', 'tennis'],
 ['female', 'Italy', 'football'],
 ['female', 'Italy', 'painting'],
 ['male', 'Spain', 'tennis'],
 ['female', 'France', 'football'],
 ['male', 'Italy', 'painting']]

In [24]:
pd.DataFrame(X, columns=['gender', 'locations', 'hobby'])

,gender,locations,hobby
0,male,Spain,tennis
1,female,Italy,football
2,female,Italy,painting
3,male,Spain,tennis
4,female,France,football
5,male,Italy,painting


In [25]:
enc = preprocessing.OneHotEncoder()
enc.fit(X)

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [26]:
enc.transform(X).toarray()

array([[0., 1., 0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 1., 0.]])

In [27]:
enc.get_feature_names_out()

array(['x0_female', 'x0_male', 'x1_France', 'x1_Italy', 'x1_Spain',
       'x2_football', 'x2_painting', 'x2_tennis'], dtype=object)

In [28]:
enc.transform([['male', 'Italy', 'football'],
               ['female', 'France', 'painting']]).toarray()

array([[0., 1., 0., 1., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0., 0., 1., 0.]])

In [29]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['France', 'Italy', 'Spain'], dtype=object),
 array(['football', 'painting', 'tennis'], dtype=object)]

#### with fixed categories value

In [30]:
genders = ['female', 'male']
locations = ['Spain', 'France', 'Italy']
hobbies = ['football', 'painting', 'tennis']
enc = preprocessing.OneHotEncoder(categories=[genders, locations, hobbies])

X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]

enc.fit(X)

,categories,"[['female', 'male'], ['Spain', 'France', ...], ...]"
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [32]:
enc.transform([['female', 'USA', 'football']]).toarray()

ValueError: Found unknown categories ['USA'] in column 1 during transform

In [7]:
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')

X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]
enc.fit(X)

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [8]:
enc.transform([['female', 'USA', 'football']]).toarray()

array([[1., 0., 0., 0., 0., 1., 0., 0.]])

In [9]:
enc.get_feature_names_out()

array(['x0_female', 'x0_male', 'x1_France', 'x1_Italy', 'x1_Spain',
       'x2_football', 'x2_painting', 'x2_tennis'], dtype=object)

In [10]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['France', 'Italy', 'Spain'], dtype=object),
 array(['football', 'painting', 'tennis'], dtype=object)]

In [11]:
enc = preprocessing.OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=2)

X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]
enc.fit(X)

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'infrequent_if_exist'
,min_frequency,2
,max_categories,None
,feature_name_combiner,'concat'


In [12]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['France', 'Italy', 'Spain'], dtype=object),
 array(['football', 'painting', 'tennis'], dtype=object)]

In [13]:
enc.get_feature_names_out()

array(['x0_female', 'x0_male', 'x1_Italy', 'x1_Spain',
       'x1_infrequent_sklearn', 'x2_football', 'x2_painting', 'x2_tennis'],
      dtype=object)

In [14]:
enc.transform([['female', 'USA', 'football']]).toarray()

array([[1., 0., 0., 0., 1., 1., 0., 0.]])

In [15]:
enc.inverse_transform([[1., 0., 0., 0., 1., 1., 0., 0.]])

array([['female', 'infrequent_sklearn', 'football']], dtype=object)

### OrdinalEncoder

In [16]:
enc = preprocessing.OrdinalEncoder()

In [43]:
X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]

enc.fit(X)

,categories,'auto'
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,unknown_value,None
,encoded_missing_value,nan
,min_frequency,None
,max_categories,None


In [44]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['France', 'Italy', 'Spain'], dtype=object),
 array(['football', 'painting', 'tennis'], dtype=object)]

In [45]:
enc.transform(X)

array([[1., 2., 2.],
       [0., 1., 0.],
       [0., 1., 1.],
       [1., 2., 2.],
       [0., 0., 0.],
       [1., 1., 1.]])

In [46]:
enc.transform([['female','Spain', 'football']])

array([[0., 2., 0.]])

## Discretization

Discretization (otherwise known as quantization or binning) provides a way to partition **continuous features into discrete values**. Certain datasets with continuous features may benefit from discretization, because discretization can transform the dataset of continuous attributes to one with only nominal attributes.

In [47]:
X = np.array([[ -3., 5., 15 ],
              [  0., 6., 14 ],
              [  6., 3., 11 ]])
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 3], encode='ordinal').fit(X)
est

/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


,n_bins,"[3, 2, ...]"
,encode,'ordinal'
,strategy,'quantile'
,quantile_method,'warn'
,dtype,None
,subsample,200000
,random_state,None


In [48]:
est.transform(X)

array([[0., 1., 2.],
       [1., 1., 1.],
       [2., 0., 0.]])

There are different strategies implemented in KBinsDiscretizer:

- ‘uniform’: The discretization is uniform in each feature, which means that the bin widths are constant in each dimension.

- ‘quantile’: The discretization is done on the quantiled values, which means that each bin has approximately the same number of samples.

- ‘kmeans’: The discretization is based on the centroids of a KMeans clustering procedure.

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html?highlight=kbinsdiscretizer

Feature binarization is the process of thresholding numerical features to get boolean values. This can be useful for downstream probabilistic estimators that make assumption that the input data is distributed according to a multi-variate Bernoulli distribution.

In [17]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]

binarizer = preprocessing.Binarizer().fit(X)  # threshold default value = 0
binarizer

,threshold,0.0
,copy,True


In [18]:
binarizer.transform(X)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [19]:
binarizer = preprocessing.Binarizer(threshold=1.1)

In [20]:
binarizer.transform(X)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 0.]])

## Custom transformers

Often, you will want to convert an existing Python function into a transformer to assist in data cleaning or processing.

In [53]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p, validate=True)
X = np.array([[0, 1], [2, 3]])
transformer.transform(X)

array([[0.        , 0.69314718],
       [1.09861229, 1.38629436]])

The *validate* parameter indicates that the input X array should be checked before calling func. The possibilities are:

- If False, there is no input validation.

- If True, then X will be converted to a 2-dimensional NumPy array or sparse matrix. If the conversion is not possible an exception is raised.

In [54]:
type(X)

numpy.ndarray

In [55]:
type(transformer.transform(X))

numpy.ndarray

In [56]:
df = pd.DataFrame(X)

In [57]:
type(df)

pandas.core.frame.DataFrame

In [58]:
type(transformer.transform(df))

numpy.ndarray

In [59]:
transformer = FunctionTransformer(np.log1p, validate=False)

In [60]:
type(transformer.transform(df))

pandas.core.frame.DataFrame

**Pay Attention!
The result of a transformer is typically a np.array!**

In [61]:
preprocessing.StandardScaler().fit_transform(df)

array([[-1., -1.],
       [ 1.,  1.]])

In [62]:
preprocessing.StandardScaler().fit_transform(X)

array([[-1., -1.],
       [ 1.,  1.]])